UsingIP                    2  
LongURL                    3  
ShortURL                   2  
Symbol@                    2  
Redirecting//              2  
PrefixSuffix-              2  
SubDomains                 3  
HTTPS                      3  
DomainRegLen               2  
Favicon                    2  
NonStdPort                 2  
HTTPSDomainURL             2  
RequestURL                 2  
AnchorURL                  3  
LinksInScriptTags          3  
ServerFormHandler          3  
InfoEmail                  2  
AbnormalURL                2  
WebsiteForwarding          2  
StatusBarCust              2  
DisableRightClick          2  
UsingPopupWindow           2  
IframeRedirection          2  
AgeofDomain                2  
DNSRecording               2  
WebsiteTraffic             3  
PageRank                   2  
GoogleIndex                2  
LinksPointingToPage        3  
StatsReport                2  
class                      2  

In [35]:
from datasets import load_dataset
import pandas as pd

# Define your dataset splits
splits = {
    'train': 'data/train-00000-of-00001-d8afc95a165ea87b.parquet',
    'test': 'data/test-00000-of-00001-4d6cbda5297196e7.parquet',
    'valid': 'data/valid-00000-of-00001-4e1abfe96aa382c2.parquet'
}

# Load the dataset from the Hugging Face Hub
dataset = load_dataset("kmack/Phishing_urls", split="train")

# Convert to pandas DataFrame if needed
df = dataset.to_pandas()

print(df.head())


                                                text  label
0             xenophongroup.com/montjoie/compgns.htm      0
1    www.azzali.eu/&usg=AOvVaw2phVSb_ENMrkATGNx5LQ0l      1
2                     guildmusic.edu.au/js/index.htm      1
3  memo.unexpectedrunner.com/ezxgytw4et\nholotili...      1
4  en.wikipedia.org/wiki/Category:American_televi...      0


In [36]:
import tldextract
def extrtact_tld(url):

    ext = tldextract.extract(url)
    # ext = ExtractResult(subdomain, domain, suffix)
    tld = ext.suffix    # "com", "co.uk", "org"
    sld = ext.domain    # second-level domain (example)
    return tld if tld else 'unknown'

df['tld'] = df['text'].apply(extrtact_tld)
df.head()

,text,label,tld
0,xenophongroup.com/montjoie/compgns.htm,0,com
1,www.azzali.eu/&usg=AOvVaw2phVSb_ENMrkATGNx5LQ0l,1,eu
2,guildmusic.edu.au/js/index.htm,1,edu.au
3,memo.unexpectedrunner.com/ezxgytw4et\nholotili...,1,com
4,en.wikipedia.org/wiki/Category:American_televi...,0,org


In [37]:
# Count phishing and legitimate occurrences for each TLD
tld_stats = df.groupby(['tld', 'label']).size().unstack(fill_value=0)
tld_stats.columns = ['legit_count', 'phish_count']

# Add totals and phishing rate
tld_stats['total'] = tld_stats['legit_count'] + tld_stats['phish_count']
tld_stats['phish_ratio'] = tld_stats['phish_count'] / tld_stats['total']
tld_stats = tld_stats.sort_values('phish_ratio', ascending=False)

print(tld_stats)


             legit_count  phish_count  total  phish_ratio
tld                                                      
zp.ua                  0            2      2          1.0
APP                    0            1      1          1.0
CO                     0            2      2          1.0
CO.ZA                  0            1      1          1.0
COM                    0           31     31          1.0
...                  ...          ...    ...          ...
brescia.it             1            0      1          0.0
bot                    1            0      1          0.0
boston                 1            0      1          0.0
vlaanderen             1            0      1          0.0
yokohama.jp            1            0      1          0.0

[1353 rows x 4 columns]


In [38]:
df['tld_phish_ratio'] = df['tld'].map(tld_stats['phish_ratio'].to_dict()).fillna(0.5)
df['tld_total_frequency'] = df['tld'].map(tld_stats['total'].to_dict()).fillna(1)
df.head()

,text,label,tld,tld_phish_ratio,tld_total_frequency
0,xenophongroup.com/montjoie/compgns.htm,0,com,0.395298,298952
1,www.azzali.eu/&usg=AOvVaw2phVSb_ENMrkATGNx5LQ0l,1,eu,0.516630,1353
2,guildmusic.edu.au/js/index.htm,1,edu.au,0.434505,313
3,memo.unexpectedrunner.com/ezxgytw4et\nholotili...,1,com,0.395298,298952
4,en.wikipedia.org/wiki/Category:American_televi...,0,org,0.374800,47396
